In [2]:
import os
import numpy as np
from PIL import Image
import cv2
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import re

from Predictor import *

default_path = 'C:/Users/paulz/Documents/CoronaryArteryDisease_v2'

os.chdir(default_path)
print(os.getcwd())


C:\Users\paulz\Documents\CoronaryArteryDisease_v2


In [3]:
#Alphanumerical sorting 
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

In [7]:
def evaluation(images_type,size):
    f1_list = []
    tp_list = []

    img_dir = os.path.join(os.getcwd(), 'Dataset', 'arcade', 'stenosis', 'test', 'images', 'img')
    img_files = sorted(os.listdir(img_dir), key=natural_sort_key)

    for name in img_files:
                                        #,'Dataset','arcade','stenosis','test','images','img')):
        true_mask_path = os.path.join(os.getcwd(),'Dataset','arcade','stenosis','test',images_type,'img',name)
        true_mask = img_to_array('masks',name,size)
        #print(true_mask)
        true_mask = np.where(true_mask>=0.5,1,true_mask)
        true_mask = np.where(true_mask<0.5,0,true_mask)
        
        pred_name = name[:-4] + '_pred.jpg'
        pred_mask = img_to_array('predictions_'+images_type,pred_name,size)
        pred_mask = np.where(pred_mask>=0.5,1,pred_mask)
        pred_mask = np.where(pred_mask<0.5,0,pred_mask)

        f1 = f1score(true_mask,pred_mask)
        f1_list.append(f1)
    f1_arr = np.array(f1_list)
    
    nan_mask = np.isnan(f1_arr)
    f1_arr[nan_mask] = 0
    
    
    return f1_arr

In [8]:
def f1score(y_true,y_pred):
    """
    computes f1 score between true and predicted masks
    """
    
    #compute true positive rate
    TP = np.sum(y_true*y_pred)
    #compute precision
    precision = TP/(np.sum(y_pred)+1e-7)

    #compute recall
    recall = TP/(np.sum(y_true)+1e-7)
    
    #compute f1 from precision and recall
    f1 = (2*precision*recall)/(precision+recall)

    return f1

In [9]:
size = (512,512)
f1_array_images = evaluation('images',size)

f1_array_kirsch = evaluation('images_kirsch',size)
f1_array_prewitt = evaluation('images_prewitt',size)
f1_array_sobel = evaluation('images_sobel',size)

C:\Users\paulz\AppData\Local\Temp\ipykernel_11888\1823143704.py:15: RuntimeWarning: invalid value encountered in scalar divide
  f1 = (2*precision*recall)/(precision+recall)


In [10]:
mean_f1_images = np.mean(f1_array_images)
print("Mean F1 Images:")
print(mean_f1_images)

mean_f1_kirsch = np.mean(f1_array_kirsch)
print("Mean F1 Kirsch:")
print(mean_f1_kirsch)

mean_f1_prewitt = np.mean(f1_array_prewitt)
print("Mean F1 Prewitt:")
print(mean_f1_prewitt)

mean_f1_sobel = np.mean(f1_array_sobel)
print("Mean F1 Sobel:")
print(mean_f1_sobel)

Mean F1 Images:
0.27845050961781465
Mean F1 Kirsch:
0.29458557493440174
Mean F1 Prewitt:
0.2531162193535607
Mean F1 Sobel:
0.2474583252468626


In [11]:
def histogram(array,name):
    plt.hist(array,bins=50,color='blue',edgecolor='black')

    plt.xlabel("F1 Score")
    plt.ylabel('Number of images')
    plt.title('Histogram ' + name)


In [ ]:
histogram(f1_array_images, 'Images')

In [ ]:
histogram(f1_array_kirsch, 'Kirsch')

In [ ]:
histogram(f1_array_prewitt, 'Prewitt')

In [ ]:
histogram(f1_array_sobel, 'Sobel')

In [195]:
#Quartilsabstände

Q1_images = np.percentile(f1_array_images,25)
print(Q1_images)
Q2_images = np.percentile(f1_array_images,50)
print(Q2_images)
Q3_images = np.percentile(f1_array_images,75)
print(Q3_images)

0.12751333423851227
0.21067298735669313
0.30636933776937264


In [114]:
#Quartilsabstände

Q1_Kirsch = np.percentile(f1_array_kirsch,25)
print(Q1_Kirsch)
Q2_Kirsch = np.percentile(f1_array_kirsch,50)
print(Q2_Kirsch)
Q3_Kirsch = np.percentile(f1_array_kirsch,75)
print(Q3_Kirsch)

0.14436851971031145
0.22230491721411866
0.327990779398541


In [115]:
#Quartilsabstände

Q1_Prewitt = np.percentile(f1_array_prewitt,25)
print(Q1_Prewitt)
Q2_Prewitt = np.percentile(f1_array_prewitt,50)
print(Q2_Prewitt)
Q3_Prewitt = np.percentile(f1_array_prewitt,75)
print(Q3_Prewitt)

0.11436395627915102
0.1917097144579457
0.27525590624065444


In [116]:
#Quartilsabstände

Q1_Sobel = np.percentile(f1_array_sobel,25)
print(Q1_Sobel)
Q2_Sobel = np.percentile(f1_array_sobel,50)
print(Q2_Sobel)
Q3_Sobel = np.percentile(f1_array_sobel,75)
print(Q3_Sobel)

0.11321367241374333
0.18023293210103136
0.2689933902545954


In [12]:
#Größte Unterschiede

full_array = np.zeros(300)
for i in range(len(f1_array_kirsch)):
    distances = np.zeros(7)
    distances[0] = np.abs(f1_array_images[i] - f1_array_kirsch[i])
    distances[1] = np.abs(f1_array_images[i] - f1_array_prewitt[i])
    distances[2] = np.abs(f1_array_images[i] - f1_array_sobel[i])

    distances[3] = np.abs(f1_array_kirsch[i] - f1_array_prewitt[i])
    distances[4] = np.abs(f1_array_kirsch[i] - f1_array_sobel[i])
        
    distances[5] = np.abs(f1_array_prewitt[i] - f1_array_sobel[i]) 

    distances[6] = i

    
    full_array[i] = np.max(distances[0:5])

#sorted_array = np.sort(sorted_array)[::-1]
print(np.argsort(-full_array))
print(np.sort(-full_array))

    

[260 144  49  36  29  22 122  30 124  77 105 273  16 253   8 184 117  65
 189  95 120 244  92 299 112 155 221  59 165 267  96 195  52 104 298  76
 251 208 242 256  73 180  38  79 170  33 132 106  90 154 261  57  32 294
 205  70  12 103 231  66  60 211 127 291 259 238  26 143 153 247 186 178
 204  78 192  81 176  61  62 269 101  54 274  83 107  20 275 110  68  80
 232 290 284 288  34 297  43 224  93 197 136 181 233 172 268 210  28 239
  75 218 196  97 243 156  45  23 199 102 285 121 187 258 148 207 296  37
 146  63 160 265 114  51 128  15  72 145 131 248 119 240 108 115   4  82
 191 198  40 263  35 252 286 292 149 183 100 228  41 216  86  87 293 270
 227  89  14 111 141 163  48 283  11  88 168 173 223  21  31 182 169   6
 142 123   2 167  19 140   9  94 126 236 220 226 151   5 116 225 162 222
 159 213 175  44 139 113 214  53 125  27 237   7 194  98 287 130 138  69
  25 255 254  56 257 161 206  18 266  71 171  85 190  46 229 147 289 235
 201 219 200 203 135  47 262 134  17 250 246 164 28

In [24]:
#Get max values and image ids
max_images = np.argsort(-f1_array_images)
print(max_images[:30])
print(f1_array_images[max_images[:30]])

[274 181 156 277   3 279  35 236 180  23  17  19  25 211  26 237 232 230
 123 257   7 224 288 225 291   4 223   8  33 276]
[0.72131148 0.68482665 0.67619917 0.65244745 0.64111498 0.63291361
 0.62102303 0.61628812 0.61592109 0.6036617  0.5971243  0.59535679
 0.58238039 0.56838849 0.55928646 0.55401917 0.55052858 0.54624127
 0.54261793 0.53545631 0.53081013 0.52917986 0.52857338 0.5274813
 0.52570037 0.50398964 0.50383384 0.49783784 0.48826661 0.48451467]


In [25]:
#Get max values and image ids
max_kirsch = np.argsort(-f1_array_kirsch)
print(max_kirsch[:30])
print(f1_array_kirsch[max_kirsch[:30]])

[156 223 274 279 211  35   7 276 224 232   3 296 122   4 123 236  26  25
 162 230 277 258  23  95  19 170 181 291  17  33]
[0.70638064 0.69864954 0.69014304 0.67295433 0.66620642 0.66326034
 0.65482062 0.64750329 0.62946708 0.62665263 0.62521447 0.62197067
 0.61744795 0.61530512 0.60800808 0.60087349 0.59233963 0.58900261
 0.58879298 0.58458213 0.57826918 0.57163891 0.5663132  0.56281148
 0.56190901 0.55002996 0.54911779 0.54894366 0.54859491 0.54554487]


In [26]:
#Get max values and image ids
max_prewitt = np.argsort(-f1_array_prewitt)
print(max_prewitt[:30])
print(f1_array_prewitt[max_prewitt[:30]])

[274 156 236 181  35 223  17   4 232  19 211 277   3 258  26 257  23 230
 296 123 279   7 122 264  33 290 255  66 237 225]
[0.78639406 0.734928   0.68161213 0.64995925 0.64804095 0.62012773
 0.61136954 0.60998114 0.60364604 0.58290534 0.577614   0.56965505
 0.56279886 0.55296984 0.55133871 0.54024988 0.52917505 0.52576541
 0.52573361 0.52546869 0.51981223 0.51965977 0.51290948 0.5120817
 0.5119108  0.49807553 0.48556612 0.47728709 0.47600841 0.47519483]


In [27]:
#Get max values and image ids
max_sobel = np.argsort(-f1_array_sobel)
print(max_sobel[:30])
print(f1_array_sobel[max_sobel[:30]])

[274  35 279 211 236 122 156  23   3   4 296 258  17 181 280 232 277 237
  19 224 230  26 123 216   7 288 183  86 290 255]
[0.73542111 0.64203774 0.63661593 0.63492598 0.62176166 0.6206717
 0.61377614 0.60034423 0.59037929 0.58411598 0.58276413 0.58119501
 0.57178386 0.56612486 0.56503447 0.53862894 0.53470085 0.52227313
 0.51226551 0.50812779 0.5047799  0.50160338 0.50017218 0.49453586
 0.49096341 0.48885336 0.47856483 0.46662536 0.46548177 0.46465328]
